# Configuring IP over UDP

This example notebook shows how a user can connect to a board with a manually configured IP address.

**A custom register configuration file (*register-config.yml*) is necessary for a successful connection. If you do not use a valid register configuration file, you will need to reboot your board or use another connection method (e.g. FTDI) to reconfigure the connection.**


## Naludaq Version

MinVersion: 0.33.2

In [8]:
# Print Naludaq version 
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

Naludaq version: 0.33.1


## Compatible Boards

* This example applies to all board models with an Ethernet port.

## Imports and Variables

In [9]:
from naluconfigs import get_available_models, get_configuration
from naludaq.board import Board, startup_board
from naludaq.communication import ControlRegisters
from naludaq.controllers import get_connection_controller

import os
import yaml

## Logging

The logger setup below will show additional information from the NaluDAQ package. This is useful for debugging purposes, but
may be skipped if not needed.

In [10]:
import logging


def setup_logger(level=logging.INFO):
    """Setup a basic logger."""
    logger = logging.getLogger()
    handler = logging.StreamHandler()
    handler.setFormatter(
        logging.Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s")
    )
    logger.addHandler(handler)
    logger.setLevel(level)
    suppress = [
        "naludaq.UART",
        "naludaq.FTDI",
    ]
    for name in suppress:
        logging.getLogger(name).setLevel(logging.CRITICAL)
    return logger

logger = setup_logger()

try:
    logger.debug("logger already setup")
except:
    logger = setup_logger()

## Options

The following parameters must be provided to successfully set the IP address:


In [11]:
# Determining Board Model
print(get_available_models())

['aardvarcv3', 'aardvarcv4', 'aodsv1', 'aodsv2_eval', 'asocv3', 'asocv3s', 'trbhm', 'dsa-c10-8', 'hdsocv1_evalr2', 'hdsocv2_eval', 'hdsocv2_evalr2', 'hiper', 'upac32', 'upac96', 'udc16', 'aodsoc_asoc', 'aodsoc_aods']


- **board_model**:      : *board_model*
  - Valid options can be found using the command in the previous code block.

- **workspace_folder**: *folder_name*
  - The folder where ADC data is saved. A workspace must be specified to successfully initiate a connection to the board but no data will be collected with this notebook example.

- **register_config_file**: *file_name*
  - The name of the custom register configuration file generated by this notebook. Must be a ".yaml" or ".yml" file.

- **board_addr**:  (*ip_address*, *port*)
  - This is the IP address of the board

- **recv_addr**:          (*ip_address*, *port*)
   - This is (typically) the IP address of your computer

Set these parameters in the following code block. All file and folder names are assumed to be in the same directory as the notebook.


In [12]:
################
# Board Config #
################

board_model = "hdsocv2_evalr2"
workspace_folder = "board_workspace"
register_config_file = "register-config.yml"

#############
# IP Config #
#############
board_addr = ("192.168.1.32", 4660)
recv_addr = ("192.168.1.198", 4660)


Loads the default register and clock configuration.

In [13]:
board = Board(board_model)
board.load_registers("register-config.yml")
board.load_clockfile()

Open connection to board and write *target_ip_address* to board. 

In [ ]:
workspace_path = os.path.join(os.getcwd(), workspace_folder)
board.start_server(workspace_path)
board.connect_udp(board_addr, recv_addr)

with board:
    if startup_board(board):
        ethernet_registers = [
            "eth_src_addr31_16",
            "eth_src_addr15_0",
            "eth_addr_sel"
        ]

        for register in ethernet_registers:
            print(ControlRegisters(board).read(register))

    else:
        print(f"Failed to start {board_model}")

board.disconnect()
board.disconnect_server()

print(f"Disconnected from {board_model}")


2025-08-05 17:23:14,612 naludaq.board                  [INFO  ]: Connecting UDP with board_addr=('192.168.1.32', 4660), receiver_addr=('192.168.1.198', 4660), backend_addr=None
2025-08-05 17:23:14,612 naludaq.board                  [INFO  ]: Connecting UDP with board_addr=('192.168.1.32', 4660), receiver_addr=('192.168.1.198', 4660), backend_addr=None
2025-08-05 17:23:14,653 naludaq.board                  [INFO  ]: Already connected.
2025-08-05 17:23:14,653 naludaq.board                  [INFO  ]: Already connected.
2025-08-05 17:23:14,693 naludaq.board_inits            [INFO  ]: Init board - hdsocv2_evalr2
2025-08-05 17:23:14,693 naludaq.board_inits            [INFO  ]: Init board - hdsocv2_evalr2
2025-08-05 17:23:14,696 naludaq.init_hdsocv2           [INFO  ]:  Running: _fpga_init ###########################################################
2025-08-05 17:23:14,696 naludaq.init_hdsocv2           [INFO  ]:  Running: _fpga_init ###########################################################


3
3


2025-08-05 17:23:15,076 naludaq.init_hdsocv2           [INFO  ]:  Running: _program_clock #######################################################
2025-08-05 17:23:15,076 naludaq.init_hdsocv2           [INFO  ]:  Running: _program_clock #######################################################
2025-08-05 17:23:15,082 naludaq.controllers.clock.si5341_controller [INFO  ]: Using si5341 config file C:\Users\plegaspi\Documents\nalu-core\naluconfigs\src\naluconfigs\data\clocks\Si5341-RevD-HDSoCv2_31p25-SYSclk_15p625-SSTclk_200-GccClk_312p5-TxClk_62p5-TxDivClk-Registers.txt
2025-08-05 17:23:15,082 naludaq.controllers.clock.si5341_controller [INFO  ]: Using si5341 config file C:\Users\plegaspi\Documents\nalu-core\naluconfigs\src\naluconfigs\data\clocks\Si5341-RevD-HDSoCv2_31p25-SYSclk_15p625-SSTclk_200-GccClk_312p5-TxClk_62p5-TxDivClk-Registers.txt
2025-08-05 17:23:21,828 naludaq.controllers.clock.si5341_controller [INFO  ]: si5341 init complete.
2025-08-05 17:23:21,828 naludaq.controllers.clock.

TimeoutError: [Status 408, ID 13] timed out while waiting for a response

## Creating a Register Configuration File

This custom configuration file must be loaded for future connections to the IP address configured using this notebook. Refer to connecting-to-new-ip.ipynb for usage example.

In [ ]:
register_config = get_configuration(board_model)
register_config["registers"]["control_registers"]["eth_dest_addr31_16"]["value"] = eth_src_addr31_16
register_config["registers"]["control_registers"]["eth_dest_addr15_0"]["value"] = eth_src_addr15_0
register_config["registers"]["control_registers"]["eth_addr_sel"]["value"] = eth_addr_sel

with open(register_config_file, "w") as file:
    yaml.dump(register_config, file)

print(f"Saved register configuration to {register_config_file}")

Saved register configuration to  register-config.yml
